In [1177]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score
stemmer=PorterStemmer()
import spacy 
nlp= spacy.load("en_core_web_sm")

In [1178]:
import re

In [1179]:
data=pd.read_csv("tweets.csv")
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [1180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   label   7920 non-null   int64 
 2   tweet   7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 185.8+ KB


In [1181]:
data.isna().sum()

id       0
label    0
tweet    0
dtype: int64

In [1182]:
data.drop(['id'],axis=1,inplace=True)
data.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...
2,0,We love this! Would you go? #talk #makememorie...
3,0,I'm wired I know I'm George I was made that wa...
4,1,What amazing service! Apple won't even talk to...


In [1183]:
data["label"].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [1184]:
balance = 2026
data_negetive = data[data["label"]==0].sample(balance, random_state=0)
data_positive = data[data["label"]==1].sample(balance, random_state=0)

In [1185]:
data_balanced = pd.concat([data_negetive,data_positive],axis=0)

In [1186]:
data_balanced["label"].value_counts()

0    2026
1    2026
Name: label, dtype: int64

In [1187]:
data_balanced.head()

,label,tweet
7335,0,love you<3 #friends #hollister #mall #shopping...
180,0,"@nobauerm - lets get you hooked up, fast. w an..."
1028,0,@JocelynB81 that hitler is always up to some f...
6136,0,Gotcha! Thanks mom! You're the BEST!!! #samsun...
7142,0,Sweet 15th! Love u!!!!!!! #birthday #dress #15...


In [1188]:
data_balanced["tweet"][180]

'@nobauerm - lets get you hooked up, fast. w an #Iphone. U will fall in . I promise.'

In [1189]:
import contractions

In [1190]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
stop_words.remove("not")
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's'

In [1191]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [1192]:
# def remove_punctuation(tweet):
#     tweet_nonpunct ="".join([new_tweet for new_tweet in tweet if new_tweet not in string.punctuation])
#     return tweet_nonpunct

In [1193]:
# data_balanced["tweet"]=data_balanced["tweet"].apply(remove_punctuation,)
data_balanced["tweet"][180]

'@nobauerm - lets get you hooked up, fast. w an #Iphone. U will fall in . I promise.'

In [1194]:
#هشتاج
#url
#link
#منشن

In [1223]:
# for token in data_balanced:
#     if token.pos_ not in ["x"]:
#         filtered_text = [token]
#     clean_text=' '.join(filtered_text)

def cleaner(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"(#\w+)", "", text)
    text = re.sub(r"@[A-Za-z0-9]+", "", text)
    text = re.sub(r" +", " ", text)
    text = contractions.fix(text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    corrected_tokens = [spell.correction(token) for token in tokens]
    filtered_tokens = [token for token in tokens if len(token)>2]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

In [ ]:
data_balanced["tweet"] = data_balanced["tweet"].apply(cleaner)

In [ ]:
data_balanced.isnull().sum()

In [ ]:
# # def empty(text):
# #     doc = nlp(text)
# #     filtered_tokens = [token.text for token in doc if token.pos_ not in ["X"]]
# #     cleaned_text = ' '.join(filtered_tokens)
# #     return cleaned_text


In [ ]:
# data_balanced["tweet"] = data_balanced["tweet"].apply(correct_spelling)

In [ ]:
data_balanced.head()

In [ ]:
# from spellchecker import SpellChecker
# spell = SpellChecker()
# def correct_spelling(text):
#     corrected_text = []
#     unknown_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in unknown_words:
#             corrected_text.append(word)  
#         else:
#             corrected_word = spell.correction(word)
#             corrected_text.append(corrected_word)
#     return " ".join(corrected_text)

In [ ]:
# data_balanced['tweet'] = data_balanced['tweet'].apply(remove_unknown_words)

In [ ]:
data_balanced.head()

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=countVectorizer
# x=cv_fit_transform(data_balanced['tweet'])

In [ ]:
# def extract_unique_named_entities(text):
#     doc = nlp(text)
#     unique_entities = set([ent.text for ent in doc.ents])
#     return unique_entities

In [ ]:
# def extract_named_entities(text):
#     doc = nlp(text)
#     entities = [ent.label_ for ent in doc.ents]
#     return entities

In [ ]:
# def display_unique_named_entities(entities):
#     print("Unique Named Entities:")
#     for entity in entities:
#         print(entity)

In [ ]:
# data_balanced['tweet'].apply(extract_named_entities).unique()

In [ ]:
# entities_to_remove = ['PERSON', 'DATE', 'NORP', 'CARDINAL', 'GPE', 'QUANTITY',
#                       'TIME', 'ORDINAL', 'ORG', 'PRODUCT', 'FAC', 'PERCENT', 'MONEY',
#                       'LANGUAGE', 'LOC', 'LAW']

In [ ]:
# def remove_named_entities(text):
#     doc = nlp(text)
#     tokens = [token.text for token in doc if token.ent_type_ not in entities_to_remove]
#     return ' '.join(tokens)

In [ ]:
# data_balanced['tweet'] = data_balanced['tweet'].apply(remove_named_entities)

In [ ]:
# data_balanced["tweet"].apply(extract_named_entities)

In [ ]:
# def remove_org_entities(text):
#     doc = nlp(text)
#     filtered_tokens = [token.text for token in doc if token.ent_type_ not in ['ORG','']]
#     cleaned_text = ' '.join(filtered_tokens)
#     return cleaned_text

In [ ]:
# data_balanced["tweet"] = data_balanced["tweet"].apply(remove_org_entities)

In [ ]:
# data_balanced["tweet"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [ ]:
y=data_balanced['label']
x=data_balanced['tweet']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
x_train_vectorized=tfidf.fit_transform(x_train)
x_test_vectorized = tfidf.transform(x_test)
x_train_vectorized

In [ ]:
from sklearn.linear_model import LogisticRegression
regressor=LogisticRegression()
regressor.fit(x_train_vectorized,y_train)

In [ ]:
y_predict = regressor.predict(x_test_vectorized)

In [ ]:
accuracy=accuracy_score(y_test,y_predict)
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm